In [1]:
from pyspark.sql import SparkSession
from random import randint
from pyspark.sql.types import IntegerType, StructField, StructType
import pyspark.sql.functions as F
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.layers import Dense
import numpy as np
#from sklearn.model_selection import train_test_split

In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable

In [3]:
spark = SparkSession.builder.appName('SparkTitanic.com').getOrCreate()

Reading the data

In [4]:
train_df = spark.read.option("inferSchema", True)\
    .option("header", True)\
    .csv('./input/titanic/train.csv')

Cleaning the data

In [5]:
train_df = train_df.select("Sex", "Age", "Survived")
train_df = train_df.where(train_df["Age"].isNotNull())
train_df = train_df.where(train_df["Sex"].isNotNull())
train_df = train_df.where(train_df["Survived"].isNotNull())
train_df = train_df.withColumn("Sex", F.when(F.col("Sex")==F.lit("male"), 0.).otherwise(1.) )

models

In [6]:
x_train = list(map(lambda x: list(x), train_df.select("Sex", "Age").collect()))
x_train = np.array([[x[0]*0.5, x[1]*0.01] for x in x_train])
y_train = list(map(lambda x: list(x)[0], train_df.select("Survived").collect()))
y_train = to_categorical(y_train,2)
#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.30, random_state=40)
model = Sequential()
model.add(Dense(32, input_dim=2, activation='tanh'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=5, epochs=12, shuffle=True, verbose=0)
#score = model.evaluate(x_test, y_test, batch_size=20, verbose=0)
#print(score)

Prediction

In [7]:
def prediction(row):
    if row.Sex == 'female':
        Sex = 1.
    elif row.Sex == 'male':
        Sex = 0.
    else:
        Sex = randint(0,1)*1.
    Age = row.Age
    if Age == None:
        Age = randint(0,100)
    Age = Age*0.01  
    p = model.predict([ [Sex, Age] ])[0]
    return int(np.argmax(p))

In [8]:
test_df = spark.read.option("inferSchema", True)\
    .option("header", True)\
    .csv('./input/titanic/test.csv')

In [9]:
output_df = spark.createDataFrame(spark.sparkContext.parallelize([]), StructType([
    StructField('PassengerId', IntegerType(), True),
    StructField('Survived', IntegerType(), True)
]))

In [10]:
for row in test_df.collect():
    output_df = output_df.union( spark.sparkContext.parallelize([(row.PassengerId, prediction(row) )]).toDF(['PassengerId', 'Survived']) )

Output

In [ ]:
output_df.toPandas().to_csv('./componentA.csv', index=False)